### Setup


In [2]:
import os
from sklearn.preprocessing import MinMaxScaler
from ipyleaflet import Map, LayerGroup, LayersControl, LegendControl
from ipywidgets import Layout

os.chdir(os.path.expanduser("../../"))
from notebooks.definitions.map import MapChart, MapChartGroup, MapDefs

scaler = MinMaxScaler(feature_range=(0, 1))
from app.database import get_connection
import pandas as pd
from app.utils.object import list_of_objects_to_dict
from app.config.column_mapping import ipcc_to_gir_code
import datetime

defs = MapDefs(scaler)
client = get_connection()
if not client.is_connected():
    await client.connect()

m = Map(
    center=(37.5519, 126.9918), zoom=10, layout=Layout(width="100%", height="700px")
)

### Load data


In [3]:
gir_4_calc = await client.iemissiondata.find_many(
    where={"source": "calc:gir4", "regionUid": {"not": None}},
    include={
        "region": True
    }
)

gir_1_calc = await client.iemissiondata.find_many(
    where={"source": "calc:gir1", "regionUid": {"not": None}},
    include={
        "region": True
    }
)


gir_1 = await client.iemissiondata.group_by(
    where={
        "source": {"not": "calc:*"},
        "categoryName": {"in": list(ipcc_to_gir_code.values())},
        "periodStartDt": datetime.datetime(2020, 1, 1),
        "periodEndDt": datetime.datetime(2020, 12, 31),
        "latitude": {"not": None},
    },
    by=["categoryName", "regionName", "source", "latitude", "longitude", ],
    sum={"emissionTotal": True},
)

gir_1_df = pd.DataFrame.from_records(gir_1)
gir_1_df["emissionTotal"] = gir_1_df["_sum"].apply(
    lambda x: x.get("emissionTotal") / 1000 if x.get("emissionTotal") else 0
)

# gir_4_group_df = pd.DataFrame.from_records(gir_4_group)
# gir_1_group_df = pd.DataFrame.from_records(gir_1_group)
# print(gir_1_df["_sum"].head(10))
gir_4_calc_df = pd.DataFrame(list_of_objects_to_dict(gir_4_calc))
gir_4_calc_ungroupped = gir_4_calc_df.copy()
gir_4_calc_df['latitude'] = gir_4_calc_df.apply(lambda x: x['region'].latitude, axis=1)
gir_4_calc_df['longitude'] = gir_4_calc_df.apply(lambda x: x['region'].longitude, axis=1)
gir_1_calc_df = pd.DataFrame(list_of_objects_to_dict(gir_1_calc))
gir_1_calc_df['latitude'] = gir_1_calc_df.apply(lambda x: x['region'].latitude, axis=1)
gir_1_calc_df['longitude'] = gir_1_calc_df.apply(lambda x: x['region'].longitude, axis=1)
gir_4_calc_groupped = gir_4_calc_df.groupby(["regionUid"], as_index=False).agg( {
    "emissionTotal": "sum", "latitude": "mean", "longitude": "mean", "regionName": "first"
} )
gir_1_calc_groupped = gir_1_calc_df.groupby(by=["regionUid"], as_index=False).agg( {
    "emissionTotal": "sum", "latitude": "mean", "longitude": "mean", "regionName": "first"
} )

gir_1_groupped = gir_1_df.groupby(by=["regionName"], as_index=False).agg( {
    "emissionTotal": "sum", "latitude": "mean", "longitude": "mean"
})

# gir_1_calc_groupped["emissionTotal"] = gir_1_calc_groupped["emissionTotal"].div(1000)



### Prepare Map


In [4]:
message_factories = "{categoryName}: {emissionTotal} Ggrams CO2eq <br> {regionName} <br> {source}"
factory_layer_data = defs.single(data=gir_4_calc_ungroupped, scale=True, bubble_message=message_factories)
factory_layer = LayerGroup(layers=factory_layer_data, name="factory")
chart_group = MapChartGroup()
data = [
    {"name": "GIR1", "data": gir_1_groupped, "colour": "red"},
    {"name": "GIR4_calc", "data": gir_4_calc_groupped, "colour": "blue"},
    {"name": "GIR1_calc", "data": gir_1_calc_groupped, "colour": "green"},
]

message_district = "{regionName} <br> {emissionTotal} Ggrams CO2eq"
group_layer = defs.group(
    data=data,
    chart_group=chart_group,
    reference_data_name="GIR1",
    reference_value_name="emissionTotal",
    bubble_message=message_district,
)
print(group_layer)
district_layer = LayerGroup(
    layers=(
        chart_group.group1.get("data")
        + chart_group.group2.get("data")
        + chart_group.group3.get("data")
    ),
    name="district",
)
m.add(factory_layer)
m.add(district_layer)
control = LayersControl(position="topright")
m.add_control(control)
legend_data = {i.get("name"): i.get("colour") for i in data}
legend = LegendControl(legend_data, name="Legend", position="bottomright")
m.add_control(legend)
m

/Users/dionysiost/workspace/hanaloop/ecoloop-ghgia/.venv/lib/python3.11/site-packages/jupyter_client/session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/Users/dionysiost/workspace/hanaloop/ecoloop-ghgia/notebooks/definitions/map.py:188: FutureWarning: Passing a DataFrame to DataFrame.from_records is deprecated. Use set_index and/or drop to modify the DataFrame instead.
  df = pd.DataFrame.from_records(data=data)
/Users/dionysiost/workspace/hanaloop/ecoloop-ghgia/notebooks/definitions/map.py:188: FutureWarning: Passing a DataFrame to DataFrame.from_records is deprecated. Use set_index and/or drop to modify the DataFrame instead.
  df = pd.DataFrame.from_records(data=data)
/Users/dionysiost/workspace/hanaloop/ecoloop-ghgia/notebooks/definitions/map.py:188: FutureWarning: Passing a DataFrame 

None


Map(center=[37.5519, 126.9918], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…